In [1]:
import pandas as pd 

#### Merging Files

In [ ]:

file1 = pd.read_csv('aggregated_results_by_ds__pool_37687290__2023_02_11.tsv', sep='\t')
file2 = pd.read_csv('aggregated_results_by_ds__pool_37806285__2023_02_18.tsv', sep='\t')
file3 = pd.read_csv('aggregated_results_by_ds__pool_37939772__2023_02_27.tsv', sep='\t')

#### Testing the common sentences between files

In [ ]:
file1.shape

(91, 4)

In [ ]:
file2.shape

(600, 4)

In [ ]:
file3.shape

(600, 4)

In [ ]:
with open('aggregated_results_by_ds__pool_37687290__2023_02_11.tsv', 'r', encoding='utf-8') as f1, open('aggregated_results_by_ds__pool_37939772__2023_02_27.tsv', 'r', encoding='utf-8') as f2:
    sentences1 = set(line.strip().split('\t')[0] for line in f1)
    sentences2 = set(line.strip().split('\t')[0] for line in f2)

shared_sentences = sentences1.intersection(sentences2)

if shared_sentences:
    print("The following sentences are present in both files:")
    for sentence in shared_sentences:
        print(sentence)
else:
    print("There are no sentences in common between the two files.")


The following sentences are present in both files:
Китай тому пример , да и не только он .
Они раскрывались поутру в траве и под вечер увядали .
Мог иметься в виду и слесарь , и столяр .
Потому что хорошее должно быть и в виде приятного .
Сотрудница туалета закатила глаза и заговорила тихо и быстро .
Бухгалтеры звонят в редакцию и просят выяснить позицию фискалов .
Дело здесь не в оборудовании , а в другом .
Бесконечные стычки и распри , дележ и взаимные наговоры .
Госпиталь формировался в Курске от союза земств и городов .
Каждый человек рождается и умирает в своем энергетическом поле .
Пытался он внести и новые предложения в производство обуви .
Сейчас единства в этом вопросе в России не существует .
Секретарь вытаращил глаза на арестанта и не дописал слова .
Да и налоги в международных водах платить не надо .
Отец в первый же понедельник ушел добровольцем в ополчение .
Остальное - в основном всякая сборочная и ремонтная мелочь .
Да и обнаружилось это в тех авариях не сразу .
Национа

In [ ]:
print("the common sntences between 2023_02_11.tsv and 2023_02_18.tsv : ", len(shared_sentences))

the common sntences between 2023_02_11.tsv and 2023_02_18.tsv :  46


In [ ]:
print("the common sntences between 2023_02_11.tsv and 2023_02_27.tsv : ", len(shared_sentences))

the common sntences between 2023_02_11.tsv and 2023_02_27.tsv :  47


In [ ]:


annotated_data = pd.concat([file2, file3], axis = 0)
annotated_data = annotated_data.drop(annotated_data.columns[-1], axis=1)#annotated_data.iloc[:, :-1]
annotated_data.dropna(inplace = True, axis=1)
#annotated_data.to_csv('annotated_data.tsv', sep='\t', index=False, header = False)
annotated_data.columns = 'text cat_s confidence'.split()
annotated_data['category'] = annotated_data.cat_s.apply(lambda x: int(x[1]))
annotated_data = annotated_data.drop(annotated_data.columns[1], axis=1)

In [ ]:
annotated_data.shape

(1200, 3)

In [ ]:
annotated_data.head()

,text,confidence,category
0,"Разглядывая их под микроскопом , мы увидим при...",100.00%,4
1,"Иногда талантливые люди , не найдя себя в наук...",100.00%,4
2,Кандидаты в депутаты от общественных организац...,100.00%,6
3,"Главное , чтобы "" социальные лифты "" не остана...",100.00%,6
4,Франция признает российских студентов и не при...,100.00%,2


In [ ]:
#annotated_data.to_csv('full_annotated_data.tsv', index=False)

# Extracting the debendency tree of the selected sentences, which are 1200 sentences, from the corpus

### Mapping [Russian dataset]


In [ ]:
df = pd.read_csv('full_annotated_data.tsv')

df.head()

,text,confidence,category
0,"Разглядывая их под микроскопом , мы увидим при...",100.00%,4
1,"Иногда талантливые люди , не найдя себя в наук...",100.00%,4
2,Кандидаты в депутаты от общественных организац...,100.00%,6
3,"Главное , чтобы "" социальные лифты "" не остана...",100.00%,6
4,Франция признает российских студентов и не при...,100.00%,2


In [ ]:
import csv
sentence_to_columns = {}
sentences_tsv = []
OUTPUT_tsv = []
CONFIDENCE_tsv = []
with open('full_annotated_data.tsv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        sentences_tsv.append(row[0])
        OUTPUT_tsv.append(row[1])
        CONFIDENCE_tsv.append(row[2])
        sentence_to_columns[row[0]] = (row[1], row[2])

In [ ]:
sentences = sentences_tsv[1:]
OUTPUT = OUTPUT_tsv[1:]
CONFIDENCE = CONFIDENCE_tsv[1:]

In [ ]:
#len(sentences)
#len(OUTPUT)
len(CONFIDENCE)

1291

In [ ]:
%%capture

!pip install nltk conllu
import nltk
import conllu

from google.colab import drive
drive.mount('/content/drive')
data_train = "/content/drive/MyDrive/Ru_Syntagrus_Dataset/ru_syntagrus-ud-train.conllu"

In [ ]:
import pandas as pd 
# Read the UD Russian SynTagRus dataset in CONLU format
with open(data_train, "r") as file:
    data = conllu.parse(file.read())

In [ ]:
filtered_data = []
for sent in data:
    # Get the text of the sentence
    text = " ".join([token["form"] for token in sent])
    if text in sentences:
        filtered_data.append(sent)

In [ ]:
# Write the filtered sentences to a new CONLL-U file

with open('full_annotated_sentence.conllu', 'w') as f:
      f.writelines([sentence.serialize() + "\n" for sentence in filtered_data])

In [ ]:
## Test the new dataset 
with open("full_annotated_sentence.conllu", "r") as file:
    data = conllu.parse(file.read())

In [ ]:
data[0]

TokenList<За, неделю, начальник, управления, собирался, позвонить, Савельеву, ,, который, занимал, теперь, такой, же, пост, ,, как, и, Семен, Еремеевич, в, соседней, области, ,, и, узнать, о, деловых, качествах, Ефимовой, ., metadata={sent_id: "2003Anketa.xml_66", text: "За неделю начальник управления собирался позвонить Савельеву, который занимал теперь такой же пост, как и Семен Еремеевич в соседней области, и узнать о деловых качествах Ефимовой."}>

In [ ]:
len(data)

1200

### Mapping [italian dataset]


In [1]:
#!pip install nltk conllu

In [2]:
import nltk
import conllu
import pandas as pd 
import re
import string


In [3]:
df = pd.read_csv('/home/jimmy/Downloads/IU/Thesis/Github/data/complexity_datasets/complexity_ds_it.csv')

df.head()

,ID,SENTENCE,judgement1,judgement2,judgement3,judgement4,judgement5,judgement6,judgement7,judgement8,...,judgement11,judgement12,judgement13,judgement14,judgement15,judgement16,judgement17,judgement18,judgement19,judgement20
0,951583956,"Quanto alla camminata incerta, va attribuita, ...",3,7,4,2,5,3,2,5,...,3,6,3,3,4,4,4,7,2,3
1,951584097,"Campione di rugby una delle vittime, un altro ...",2,4,3,2,3,3,3,1,...,1,1,2,1,1,3,2,2,2,3
2,951583629,Costo dalle 100.000 alle 150.000 il mq.,2,3,2,2,1,1,2,1,...,1,3,1,3,1,1,1,1,1,1
3,951583242,"A Valona, dove ieri è stata convocata un'altra...",1,5,3,4,2,2,1,3,...,1,3,4,2,3,1,4,3,5,3
4,951583156,"Il mito di Allende, più di 14 anni dopo la sua...",4,5,6,2,3,5,4,5,...,4,5,7,2,4,3,5,7,3,2


In [4]:
it_sentences = df['SENTENCE'].tolist()

In [5]:
it_sentences[:5]

['Quanto alla camminata incerta, va attribuita, fino a prova contraria, alla causa di sempre:',
 'Campione di rugby una delle vittime, un altro ha le gambe tranciate.',
 'Costo dalle 100.000 alle 150.000 il mq.',
 "A Valona, dove ieri è stata convocata un'altra manifestazione di protesta cui hanno partecipato 5.000 persone, l'atmosfera è meno elettrica.",
 'Il mito di Allende, più di 14 anni dopo la sua tragica morte, comincia ad affievolirsi anche nella sinistra cilena?']

In [6]:
len(it_sentences)

1122

In [7]:
it_ds = "/home/jimmy/Downloads/IU/Thesis/Universal Dependencies 2.10/ud-treebanks-v2.10/UD_Italian-ISDT/it_isdt-ud-train.conllu"


In [8]:

def normalize_sentence(sentence):
    # Remove spaces around punctuation marks
    sentence = re.sub(r'\s+([.,;:!?-])', r'\1', sentence)
    
    # Remove punctuation, convert to lowercase, and replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', sentence.translate(str.maketrans("", "", string.punctuation)).lower().strip())


In [9]:
with open(it_ds, "r") as file:
    it_data = conllu.parse(file.read())

In [10]:
len(it_data)

13121

In [11]:
it_data[3]

TokenList<Inconsueto, allarme, alla, a, la, Tate, Gallery, :, metadata={sent_id: "isst_tanl-5", text: "Inconsueto allarme alla Tate Gallery:"}>

In [12]:
# Create a dictionary to map sentences to their corresponding dependency tree
it_trees = {normalize_sentence(str(sentence.metadata['text'])): sentence for sentence in it_data}


In [13]:
# Extract the dependency tree for each sentence
normalized_it_sentences = [normalize_sentence(sentence) for sentence in it_sentences]


In [14]:
# Extract the dependency tree for each normalized sentence
it_dependency_trees = [it_trees[normalized_sentence] for normalized_sentence in normalized_it_sentences if normalized_sentence in it_trees]


In [15]:
for tree in it_dependency_trees[:5]:
    print(tree)
    print()

TokenList<Quanto, alla, a, la, camminata, incerta, ,, va, attribuita, ,, fino, a, prova, contraria, ,, alla, a, la, causa, di, sempre, :, metadata={sent_id: "isst_tanl-282", text: "Quanto alla camminata incerta, va attribuita, fino a prova contraria, alla causa di sempre:"}>

TokenList<Campione, di, rugby, una, delle, di, le, vittime, ,, un, altro, ha, le, gambe, tranciate, ., metadata={sent_id: "isst_tanl-1248", text: "Campione di rugby una delle vittime, un altro ha le gambe tranciate."}>

TokenList<Costo, dalle, da, le, 100.000, alle, a, le, 150.000, il, mq, ., metadata={sent_id: "isst_tanl-3128", text: "Costo dalle 100.000 alle 150.000 il mq."}>

TokenList<A, Valona, ,, dove, ieri, è, stata, convocata, un', altra, manifestazione, di, protesta, cui, hanno, partecipato, 5.000, persone, ,, l', atmosfera, è, meno, elettrica, ., metadata={sent_id: "tut-2252", text: "A Valona, dove ieri è stata convocata un'altra manifestazione di protesta cui hanno partecipato 5.000 persone, l'atmosfera

In [16]:
len(it_dependency_trees)

1081

In [17]:

#!pip install python-Levenshtein

In [18]:
import Levenshtein

def find_closest_sentence(sentence, sentence_tree_map):
    distances = [(Levenshtein.distance(sentence, tree_sentence), tree_sentence) for tree_sentence in sentence_tree_map]
    closest_sentence = min(distances, key=lambda x: x[0])[1]
    return sentence_tree_map[closest_sentence]

In [19]:
# find the closest match in corpus for each missing sentence based on the Levenshtein distance and append the corresponding dependency tree to the it_dependency_trees list.
it_dependency_trees = []


for index, normalized_sentence in enumerate(normalized_it_sentences):
    if normalized_sentence in it_trees:
        it_dependency_trees.append(it_trees[normalized_sentence])
    else:
        
        closest_tree = find_closest_sentence(normalized_sentence, it_trees)
        it_dependency_trees.append(closest_tree)
        print('the original text : ', normalized_sentence,'\n')
        print('closest sentence : ', closest_tree)
        print("")

the original text :  i farmaci anti rigetto hanno un effetto di indebolimento sulle difese immunitarie e il trapiantato è più esposto a infezioni influenze 

closest sentence :  TokenList<I, farmaci, anti, -, rigetto, hanno, un, effetto, di, indebolimento, sulle, su, le, difese, immunitarie, e, il, trapiantato, è, più, esposto, a, infezioni, ,, influenze, ., metadata={sent_id: "isst_tanl-3919", text: "I farmaci anti-rigetto hanno un effetto di indebolimento sulle difese immunitarie e il trapiantato è più esposto a infezioni, influenze."}>

the original text :  sarebbero numerosi i dipendenti del servizio pubblico afflitti dal vizietto di pigiare un potroppo il piede sullacceleratore 

closest sentence :  TokenList<sarebbero, numerosi, i, dipendenti, del, di, il, servizio, pubblico, afflitti, dal, da, il, ", vizietto, ", di, pigiare, un, po', troppo, il, piede, sull', su, l', acceleratore, ., metadata={sent_id: "isst_tanl-2780", text: "sarebbero numerosi i dipendenti del servizio pubbli

the original text :  potente e istrionico stoichkov è andato in gol al 34 del primo tempo finalizzando alla perfezione una combinazione zola inzaghi 

closest sentence :  TokenList<Potente, e, istrionico, Stoichkov, è, andato, in, gol, al, a, il, 34', del, di, il, primo, tempo, finalizzando, alla, a, la, perfezione, una, combinazione, Zola, Inzaghi, ., metadata={sent_id: "isst_tanl-2365", text: "Potente e istrionico Stoichkov è andato in gol al 34'del primo tempo finalizzando alla perfezione una combinazione Zola Inzaghi."}>

the original text :  roosevelt ci veniva regolarmente e a tuttoggi alcune società di assicurazione prima di stipulare una polizza vita cominciano a esigere un accreditatissimo stato di salute mayo 

closest sentence :  TokenList<Roosevelt, ci, veniva, regolarmente, e, ,, a, tutt', oggi, ,, alcune, società, di, assicurazione, ,, prima, di, stipulare, una, polizza, -, vita, ,, cominciano, a, esigere, un, accreditatissimo, ", stato, di, salute, Mayo, ", ., metadata={

the original text :  cè un popiù di vita e anche questa è una consuetudine nei quartieri periferici sono più numerosi quelli che non possono andare in vacanza 

closest sentence :  TokenList<C', è, un, po', più, di, vita, -, e, anche, questa, è, una, consuetudine, -, nei, in, i, quartieri, periferici, ,, sono, più, numerosi, quelli, che, non, possono, andare, in, vacanza, ., metadata={sent_id: "isst_tanl-2840", text: "C'è un po' più di vita - e anche questa è una consuetudine - nei quartieri periferici, sono più numerosi quelli che non possono andare in vacanza."}>

the original text :  litalia ha già eguagliato il bottino di roma87 quando gli azzurri si imposero con panetta 3000 siepi e damilano 20 km di marcia 

closest sentence :  TokenList<L', Italia, ha, già, eguagliato, il, bottino, di, Roma, '87, ,, quando, gli, azzurri, si, imposero, con, Panetta, (, 3.000, siepi, ), e, Damilano, (, 20, km, di, marcia, ), ., metadata={sent_id: "isst_tanl-2251", text: "L'Italia ha già eguagliato

In [20]:

print(f"Number of sentences in complexity_ds_it.csv: {len(it_sentences)}")
print(f"Number of extracted dependency trees: {len(it_dependency_trees)}")


Number of sentences in complexity_ds_it.csv: 1122
Number of extracted dependency trees: 1122


In [21]:
with open('it_dependency_trees.conllu', 'w') as file:
    for tree in it_dependency_trees:
        file.write(tree.serialize())
        file.write("\n")

In [22]:
output_path = '/home/jimmy/Downloads/IU/Thesis/Github/notebooks/it_dependency_trees.conllu'
with open(output_path, "r") as file:
    data = conllu.parse(file.read())

In [23]:
len(data)

1122